In [ ]:
!pip install langchain
!pip install transformers
!pip install bitsandbytes accelerate
!pip install -U langchain-community
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login(token="-------")#provide huggingface token


4bit quantization [requires a gpu]

In [3]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig



nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_id = "Orenguteng/Llama-3-8B-Lexi-Uncensored"# use compatible a model

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                 model_id,quantization_config=nf4_config  ## remove quantization_config if running on CPU
                 )
pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
               )

In [ ]:
hf = HuggingFacePipeline(pipeline=pipe)
while True:
  x=input(">>")
  messages = [
    {"role": "system", "content": "  Your are Pluto. you are a sarcastic and arrogant, you are good at answering questions"},#system prompt
    {"role": "user", "content": x},
]
  response = pipe(messages,max_length=150, temperature=0.7, top_p=0.9, top_k=50, num_return_sequences=1,truncation=True) 


  print(response[0]["generated_text"][len(response[0]["generated_text"])-1]['content'])